In [3]:
!nvidia-smi

import os
import warnings
from IPython.display import clear_output

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
warnings.filterwarnings('ignore')

Tue Nov  7 16:43:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   53C    P0    51W / 300W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install pytorch_lightning
clear_output()

In [5]:
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from transformers import AutoTokenizer
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
pl.seed_everything(56)

Seed set to 56


56

In [2]:
class CFG:
    wandb=False
    num_workers=12
    train_path='/notebooks/train.gz'
    hidden_size = 512
    val_split_size = 0.2
    num_labels = 2
    scheduler='cosine'
    max_epoches=15
    lr=1e-3
    min_lr=5e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=2048
    weight_decay=0.01
    gradient_accumulation_steps=1
    seed=56

In [4]:
class PLDataset(torch.utils.data.Dataset):
    def __init__(self, df,features,label):
        super().__init__()
        self.cfg = CFG()
        self.data = df
        self.features_col = features
        self.label_col = label
    def __getitem__(self, index):
        features = self.data.iloc[index][self.features_col].values.astype(np.int32)
        label = self.data.iloc[index][self.label_col]
        return features,label
    def __len__(self):
        return len(self.data)

In [5]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self,df,features,label):
        super().__init__()
        self.cfg = CFG()
        self.df = df
        self.features = features
        self.label = label
        self.train_dataset_path = self.cfg.train_path
        self.val_split_size = self.cfg.val_split_size
        self.batch_size = self.cfg.batch_size
        self.num_workers = self.cfg.num_workers
        self.is_setup = False
    def prepare_data(self):
        pass
    
    def setup(self, stage: str):
        if self.is_setup:
            return None
        self.train_df, self.val_df = train_test_split(self.df, test_size=self.val_split_size,random_state=self.cfg.seed)
        self.train_dataset = PLDataset(self.train_df,self.features,self.label)
        self.val_dataset = PLDataset(self.val_df,self.features,self.label)
       # self.predict_dataset = PLDataset(self.test_df,self.features,self.label)
        self.is_setup = True
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers)

    def predict_dataloader(self):
        return DataLoader(self.predict_dataset,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          shuffle = False)

In [28]:
class FactorizationMachine(torch.nn.Module):

    def __init__(self, reduce_sum=False):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

In [79]:
class PLModule(pl.LightningModule):
    def __init__(self,):
        super().__init__()
        self.cfg = CFG()
        self.embedings = nn.ModuleList([nn.Embedding(num_emb,emb_dim)
                                       for num_emb,emb_dim in zip(self.cfg.embed_num,self.cfg.embed_dim)])
        self.mlp = nn.Sequential(nn.Linear(self.cfg.embed_size,self.cfg.hidden_size),
                                 nn.GELU(),
                                 nn.LayerNorm(self.cfg.hidden_size),
                                 nn.Linear(self.cfg.hidden_size,1))
        self.fm = FactorizationMachine()
        self.criterion = nn.BCELoss()
        self.val_targets = []
        self.val_preds = []

    def forward(self, x):
        features = torch.cat([emb(x_i.int()) for emb,x_i in zip(self.embedings,x.T)],dim=-1)
        return torch.sigmoid(self.mlp(features).squeeze() + self.fm(features))
    
    def training_step(self, batch, _):
        x, targets = batch
        logits = self(x).to(torch.double)
        loss = self.criterion(logits, targets.to(torch.double))
        return loss
        
    def validation_step(self, batch, _):
        x, targets = batch
        logits = self(x).cpu().detach().tolist()
        self.val_targets += targets.tolist()
        self.val_preds += logits
        
    def predict_step(self, batch, _):
        x, targets = batch
        logits = self(x).cpu().detach().tolist()
        return logits
        
    def calc_metric(self):
        return roc_auc_score(self.val_targets,self.val_preds)
        
    def on_validation_epoch_end(self):
        print(self.calc_metric())
        self.val_targets, self.val_preds = [],[]
            
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(),
                                 self.cfg.lr,
                                 weight_decay=self.cfg.weight_decay,
                                 betas = self.cfg.betas
                                )

In [8]:
df = pd.read_csv("train.gz")

In [9]:
cat_features = ['hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
features = cat_features
label_col = 'click'

In [62]:
class MultyLabelEncoder():
    def __init__(self,cat_cols):
        self.cat_cols = cat_cols
    
    def fit_transform(self,df):
        self.encoders = []
        for col in tqdm(self.cat_cols):
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            self.encoders += [le]
        return df
    
    def transform(self,df):
        for col,le in tqdm(zip(self.cat_cols,self.encoders),total=len(self.cat_cols)):
            df[col] = le.transform(df[col])
        return df

In [11]:
le = MultyLabelEncoder(cat_features)
df = le.fit_transform(df)

  0%|          | 0/22 [00:00<?, ?it/s]

In [12]:
CFG.embed_num = [df[col].nunique() for col in tqdm(cat_features)]

  0%|          | 0/22 [00:00<?, ?it/s]

In [13]:
CFG.embed_dim = [64] * len(CFG.embed_num)
CFG.embed_size = sum(CFG.embed_dim)
CFG.hidden_size = CFG.embed_size * 2

In [14]:
dm = PLDataModule(df,features,label_col)

In [80]:
pl_model = PLModule()

In [81]:
logger = pl.loggers.TensorBoardLogger("tb_logs", name="text_cls")
trainer = pl.Trainer(
    accelerator="gpu",
    logger=logger,
    max_epochs=15,
    log_every_n_steps=1,
    val_check_interval=0.25
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(pl_model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                 | Params
---------------------------------------------------
0 | embedings | ModuleList           | 604 M 
1 | mlp       | Sequential           | 4.0 M 
2 | fm        | FactorizationMachine | 0     
3 | criterion | BCELoss              | 0     
---------------------------------------------------
608 M     Trainable params
0         Non-trainable params
608 M     Total params
2,434.963 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

0.4852904453543909


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
np.stack(pl_model.val_preds).shape

In [59]:
np.stack(pl_model.val_targets).shape

(4096,)